Goal: use autoencoders to reduce image noise

In [ ]:
# Exercise - embed the image in your notebook

<img src="https://miro.medium.com/max/750/1*sahNK4wy4teFA0r6tJJwKQ.png
" alt="Stock Price Demo" width="300" height=200/>

#### Import data

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist # fashion MNIST data built
#into tensorflow

# We don't need y_train and y_test
(x_train, _), (x_test, _) = fashion_mnist.load_data()

print('Max value in the x_train is', x_train[0].max())
print('Min value in the x_train is', x_train[0].min())

#### EDA

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(5, 10) 
fig.tight_layout(pad=-1)
plt.gray()
a = 0 
for i in range(5): 
  for j in range(10): 
    axs[i, j].imshow(tf.squeeze(x_test[a])) # visualise MNIST images
    axs[i, j].xaxis.set_visible(False) 
    axs[i, j].yaxis.set_visible(False) 
    a = a + 1 

#### Normalize the Image Data

In [ ]:
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

In [ ]:
# check the shape of our data
x_train.shape

In [ ]:
# we need to add an extra "dimension" to turn our image data into a tensor
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],x_train.shape[2],1)
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2],1)

In [ ]:
x_train.shape

In [ ]:
x_test.shape

#### Adding Noise to the images

In [ ]:
noise_factor = 0.4
x_train_noisy = x_train + noise_factor * tf.random.normal(shape=x_train.shape) 
x_test_noisy = x_test + noise_factor * tf.random.normal(shape=x_test.shape)

In [ ]:
# we need to normalize again as we have added noise
x_train_noisy = tf.clip_by_value(x_train_noisy, clip_value_min=0., clip_value_max=1.) 
x_test_noisy = tf.clip_by_value(x_test_noisy, clip_value_min=0., clip_value_max=1.)

In [ ]:
# check our "noisy" images

n = 5
plt.figure(figsize=(20, 8))
plt.gray()

for i in range(n):
  ax = plt.subplot(2, n, i + 1) 
  plt.title("original", size=20) 
  plt.imshow(tf.squeeze(x_test[i])) 
  plt.gray() 
  bx = plt.subplot(2, n, n+ i + 1) 
  plt.title("original + noise", size=20) 
  plt.imshow(tf.squeeze(x_test_noisy[i])) 
plt.show()

#### Building the Model

In [ ]:
from tensorflow.keras.layers import Conv2DTranspose, Conv2D, Input

class NoiseReducer(tf.keras.Model): 
  def __init__(self):

    super(NoiseReducer, self).__init__() 

    self.encoder = tf.keras.Sequential([ 
      Input(shape=(28, 28, 1)), 
      Conv2D(16, (3,3), activation='relu', padding='same', strides=2), 
      Conv2D(8, (3,3), activation='relu', padding='same', strides=2)]) 
    
    self.decoder = tf.keras.Sequential([ 
      Conv2DTranspose(8, kernel_size=3, strides=2, activation='relu', padding='same'), 
      Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='same'), 
      Conv2D(1, kernel_size=(3,3), activation='sigmoid', padding='same')]) 
  
  def call(self, x): 
    encoded = self.encoder(x) 
    decoded = self.decoder(encoded) 
    return decoded

In [ ]:
autoencoder = NoiseReducer() # sets up our autoencoder as an instance of our
# NoiseReducer class

In [ ]:
autoencoder.compile()

#### Configuring the Model

In [ ]:
# Exercise - 
# a) compile the autoencoder
# b) fit the model to the "noisy" data and
    # benchmark against the original data
    # use 10 epochs
    # shuffle the data

In [ ]:
autoencoder.compile(optimizer="adam", loss="mse")

In [ ]:
help(autoencoder.compile)

In [ ]:
autoencoder.fit(x_train_noisy,
                x_train, 
                epochs = 10,
                shuffle = True,
                validation_data=(x_test_noisy,
                x_test))

#### Reducing Image Noise with our trained autoencoder

In [ ]:
encoded_imgs = autoencoder.encoder(x_test_noisy).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs) # reconstrcuted images

In [ ]:
# plot - how do our reconstructed images look in comparison with our originals ??


n = 10 
plt.figure(figsize=(20, 7))
plt.gray()
for i in range(n): 
  # display original + noise 
  bx = plt.subplot(3, n, i + 1) 
  plt.title("original + noise") 
  plt.imshow(tf.squeeze(x_test_noisy[i])) 
  ax.get_xaxis().set_visible(False) 
  ax.get_yaxis().set_visible(False) 
  
  # display reconstruction 
  cx = plt.subplot(3, n, i + n + 1) 
  plt.title("reconstructed") 
  plt.imshow(tf.squeeze(decoded_imgs[i])) 
  bx.get_xaxis().set_visible(False) 
  bx.get_yaxis().set_visible(False) 
  
  # display original 
  ax = plt.subplot(3, n, i + 2*n + 1) 
  plt.title("original") 
  plt.imshow(tf.squeeze(x_test[i])) 
  ax.get_xaxis().set_visible(False) 
  ax.get_yaxis().set_visible(False) 

plt.show()